In [1]:
import numpy as np
from sklearn.model_selection import ShuffleSplit
from data_utils import ENTITIES, Documents, Dataset, SentenceExtractor, make_predictions
from data_utils import Evaluator
from models import build_lstm_crf_model
from gensim.models import Word2Vec

Using TensorFlow backend.
/Users/nastul/Documents/anaconda3/envs/ner/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/nastul/Documents/anaconda3/envs/ner/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/nastul/Documents/anaconda3/envs/ner/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1

In [2]:
data_dir = 'brat/'
ent2idx = dict(zip(ENTITIES, range(1, len(ENTITIES) + 1)))
idx2ent = dict([(v, k) for k, v in ent2idx.items()])

In [3]:
#载入文件
docs = Documents(data_dir=data_dir)
#分配训练集和测试集
#自动迭代，返回id
rs = ShuffleSplit(n_splits=1, test_size=20, random_state=2018)
train_doc_ids, test_doc_ids = next(rs.split(docs))
train_docs, test_docs = docs[train_doc_ids], docs[test_doc_ids]

In [4]:
docs.doc_ids

array(['11319941', '11532192', '11597317', '11604102', '11897010',
       '12079497', '12546709', '12585968', '12925238', '14609438',
       '14611657', '14624252', '14675480', '14691534', '14723793',
       '14737183', '15005800', '15018652', '15040800', '15061865',
       '15070402', '15207008', '15238161', '15314655', '15314659',
       '15320950', '15328533', '15328538', '15345036', '15492776',
       '15550985', '15560850', '15588329', '15615595', '15619330',
       '15630473', '15676071', '15760270', '15784609', '15819996',
       '15836427', '15850489', '15876356', '15882093', '15917436',
       '15921521', '15938754', '16026622', '16027110', '16098226',
       '16103912', '16109169', '16110338', '16121255', '16121256',
       '16216087', '16221973', '16255782', '16279840', '16362077',
       '16410827', '16433929', '16462940', '16504143', '16504174',
       '16507151', '16517939', '16539743', '16579849', '16611361',
       '16628246', '16670015', '16700629', '16787536', '168008

In [26]:
docs[0].doc_id

'11319941'

In [20]:
next(rs.split(docs))

(array([93, 23, 11, 65, 67, 18, 32, 30, 74, 14, 48, 57, 80, 15, 27,  8, 24,
        46, 55, 39, 61, 89, 86, 54, 50, 12,  1, 19,  4, 16,  3, 34, 10, 72,
        35, 82, 36, 58, 76, 49, 64, 68, 44, 63, 13, 70, 45, 78, 81, 83, 69,
        84, 29, 79, 71, 40, 53, 38, 26,  0, 42, 92, 22, 31, 60, 90, 88, 47,
        75, 87, 91, 25,  6, 20, 28, 21,  9]),
 array([59,  7, 85, 41, 94, 33, 17, 56, 52,  2, 77, 37,  5, 62, 73, 95, 51,
        96, 66, 43]))

In [22]:
docs

In [5]:
test_doc_ids

array([59,  7, 85, 41, 94, 33, 17, 56, 52,  2, 77, 37,  5, 62, 73, 95, 51,
       96, 66, 43])

In [6]:
train_doc_ids

array([93, 23, 11, 65, 67, 18, 32, 30, 74, 14, 48, 57, 80, 15, 27,  8, 24,
       46, 55, 39, 61, 89, 86, 54, 50, 12,  1, 19,  4, 16,  3, 34, 10, 72,
       35, 82, 36, 58, 76, 49, 64, 68, 44, 63, 13, 70, 45, 78, 81, 83, 69,
       84, 29, 79, 71, 40, 53, 38, 26,  0, 42, 92, 22, 31, 60, 90, 88, 47,
       75, 87, 91, 25,  6, 20, 28, 21,  9])

In [27]:
ent2idx

{'GO_UBERON_EXT': 1,
 'CHEBI_GO_SO_EXT': 2,
 'SO': 3,
 'CHEBI_GO_EXT': 4,
 'PR_EXT': 5,
 'CHEBI_EXT': 6,
 'PATO_SO_EXT': 7,
 'CHEBI_CHMO_EXT': 8,
 'CL_EXT': 9,
 'GO_PATO_UBERON_EXT': 10,
 'MOP': 11,
 'GO_PATO_EXT': 12,
 'GO_RO_EXT': 13,
 'OBI_SO_EXT': 14,
 'GO_EXT': 15,
 'CHEBI_PR_EXT': 16,
 'CHEBI_SO_EXT': 17,
 'CHEBI_MOP_EXT': 18,
 'CL_UBERON_EXT': 19,
 'PATO_UBERON_EXT': 20,
 'NCBITaxon_EXT': 21,
 'CHEBI_UBERON_EXT': 22,
 'UBERON': 23,
 'CHMO_UBERON_EXT': 24,
 'GO_SO_EXT': 25,
 'NCBITaxon': 26,
 'SO_EXT': 27,
 'GO_PR_EXT': 28,
 'PR': 29,
 'CL_GO_EXT': 30,
 'UBERON_EXT': 31,
 'GO_PATO_RO_EXT': 32,
 'CL': 33,
 'CHEBI_GO_PR_EXT': 34,
 'CHEBI': 35,
 'CHEMINF_GO_EXT': 36,
 'NCBITaxon_UBERON_EXT': 37,
 'GO_MOP_EXT': 38,
 'GO': 39}

In [7]:
#实体个数
num_cates = max(ent2idx.values()) + 1
#句子长度
sent_len = 64
vocab_size = 3000
#向量长度
emb_size = 100
#两头padding长度
sent_pad = 10
#抽取句子
sent_extrator = SentenceExtractor(window_size=sent_len, pad_size=sent_pad)
train_sents = sent_extrator(train_docs)
test_sents = sent_extrator(test_docs)
train_data = Dataset(train_sents, cate2idx=ent2idx)
train_data.build_vocab_dict(vocab_size=vocab_size)
test_data = Dataset(test_sents, word2idx=train_data.word2idx, cate2idx=ent2idx)
vocab_size = len(train_data.word2idx)

In [9]:
w2v_train_sents = []
for doc in docs:
    w2v_train_sents.append(list(doc.text))
    
    
w2v_model = Word2Vec(w2v_train_sents, size=emb_size)
w2v_embeddings = np.zeros((vocab_size, emb_size))
for char, char_idx in train_data.word2idx.items():
    if char in w2v_model.wv:
        w2v_embeddings[char_idx] = w2v_model.wv[char]

In [10]:
len(w2v_train_sents[0]) #第一个文章中所有的字符
#字符级的词嵌入

33532

In [38]:
w2v_model.wv

In [37]:
train_data.word2idx.items()

dict_items([('<unk>', 0), (' ', 1), ('e', 2), ('t', 3), ('i', 4), ('n', 5), ('a', 6), ('o', 7), ('s', 8), ('r', 9), ('l', 10), ('d', 11), ('c', 12), ('h', 13), ('m', 14), ('u', 15), ('p', 16), ('f', 17), ('g', 18), ('y', 19), ('b', 20), ('w', 21), ('.', 22), (',', 23), ('v', 24), ('A', 25), ('-', 26), ('1', 27), ('\n', 28), ('T', 29), ('C', 30), (')', 31), ('(', 32), ('x', 33), ('2', 34), ('P', 35), ('0', 36), ('k', 37), ('S', 38), ('F', 39), ('B', 40), ('5', 41), ('D', 42), ('M', 43), ('G', 44), ('3', 45), ('/', 46), ('I', 47), ('R', 48), ('E', 49), ('4', 50), ('N', 51), ('H', 52), ('6', 53), ('7', 54), ('z', 55), ('L', 56), ('9', 57), (']', 58), ('[', 59), ('O', 60), ('8', 61), ('K', 62), ('W', 63), ('q', 64), (';', 65), ('X', 66), ('+', 67), ('J', 68), ('U', 69), (':', 70), ('%', 71), ('–', 72), ('V', 73), ('j', 74), ('−', 75), ('Q', 76), ("'", 77), ('β', 78), ('=', 79), ('μ', 80), ('Y', 81), ('Z', 82), ('α', 83), ('′', 84), ('Δ', 85), ('<', 86), ('×', 87), ('°', 88), ('±', 89), ('*

In [11]:
#句子的长度前后都padding
seq_len = sent_len + 2 * sent_pad
model = build_lstm_crf_model(num_cates, seq_len=seq_len, vocab_size=vocab_size, 
                             model_opts={'emb_matrix': w2v_embeddings, 'emb_size': 100, 'emb_trainable': False})
model.summary()

G:\Coding\Anaconda3\envs\ner\lib\site-packages\keras_contrib-2.0.8-py3.6.egg\keras_contrib\layers\crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
G:\Coding\Anaconda3\envs\ner\lib\site-packages\keras_contrib-2.0.8-py3.6.egg\keras_contrib\layers\crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 84)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 84, 100)           15100     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 84, 512)           731136    
_________________________________________________________________
crf_1 (CRF)                  (None, 84, 40)            22200     
Total params: 768,436
Trainable params: 753,336
Non-trainable params: 15,100
_________________________________________________________________


In [12]:
train_X, train_y = train_data[:]
print('train_X.shape', train_X.shape)
print('train_y.shape', train_y.shape)

train_X.shape (50146, 84)
train_y.shape (50146, 84, 1)


In [ ]:
type(train_X)

In [13]:
train_X[0]

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1, 42,  2, 10,  2,  3,  4,  7,
        5,  1,  6,  3,  1, 47, 29, 35, 48, 27,  1, 69,  5, 11,  2,  9, 10,
        4,  2,  8,  1, 25,  3,  6, 33,  4,  6,  1,  4,  5,  1, 43,  4, 12,
        2,  1,  6,  5, 11,  1, 38, 16,  4,  5,  7, 12,  2,  9,  2, 20,  2,
       10, 10,  6,  9,  1, 25,  3,  6, 33,  4,  6,  1, 27, 41,  1,  4])

In [39]:
model.fit(train_X,train_y, batch_size=64, epochs=10)


Epoch 1/10
50146/50146 [==============================] - 334s 7ms/step - loss: 0.6775 - crf_viterbi_accuracy: 0.7987
Epoch 2/10
50146/50146 [==============================] - 338s 7ms/step - loss: 0.2435 - crf_viterbi_accuracy: 0.8867
Epoch 3/10
50146/50146 [==============================] - 330s 7ms/step - loss: 0.1326 - crf_viterbi_accuracy: 0.9243
Epoch 4/10
50146/50146 [==============================] - 333s 7ms/step - loss: 0.0892 - crf_viterbi_accuracy: 0.9416
Epoch 5/10
50146/50146 [==============================] - 330s 7ms/step - loss: 0.0658 - crf_viterbi_accuracy: 0.9516
Epoch 6/10
50146/50146 [==============================] - 327s 7ms/step - loss: 0.0503 - crf_viterbi_accuracy: 0.9590
Epoch 7/10
50146/50146 [==============================] - 330s 7ms/step - loss: 0.0386 - crf_viterbi_accuracy: 0.9643
Epoch 8/10
50146/50146 [==============================] - 338s 7ms/step - loss: 0.0284 - crf_viterbi_accuracy: 0.9689
Epoch 9/10
50146/50146 [==============================]

In [57]:
model

In [52]:
#句子的长度前后都padding
seq_len = sent_len + 2 * sent_pad
model1 = build_lstm_crf_model(num_cates, seq_len=seq_len, vocab_size=vocab_size, 
                             model_opts={'emb_matrix': w2v_embeddings, 'emb_size': 100, 'emb_trainable': False})
model1.summary()

G:\Coding\Anaconda3\envs\ner\lib\site-packages\keras_contrib-2.0.8-py3.6.egg\keras_contrib\layers\crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
G:\Coding\Anaconda3\envs\ner\lib\site-packages\keras_contrib-2.0.8-py3.6.egg\keras_contrib\layers\crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 84)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 84, 100)           15100     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 84, 512)           731136    
_________________________________________________________________
crf_2 (CRF)                  (None, 84, 40)            22200     
Total params: 768,436
Trainable params: 753,336
Non-trainable params: 15,100
_________________________________________________________________


In [54]:
from keras.models import load_model
model2 = load_model("model")

In [40]:
test_X, _ = test_data[:]
preds = model.predict(test_X, batch_size=64, verbose=True)
pred_docs = make_predictions(preds, test_data, sent_pad, docs, idx2ent)

13837/13837 [==============================] - 31s 2ms/step


In [41]:
test_data

In [42]:
test_X

array([[ 1,  1,  1, ..., 11,  1, 40],
       [ 2,  1, 17, ..., 17,  1, 53],
       [ 6,  3,  2, ...,  8, 13,  7],
       ...,
       [23,  1, 69, ...,  2,  3,  1],
       [ 1, 52, 23, ...,  1,  3, 13],
       [20,  3, 19, ...,  1,  1,  1]])

In [43]:
f_score, precision, recall = Evaluator.f1_score(test_docs, pred_docs)
print('f_score: ', f_score)
print('precision: ', precision)
print('recall: ', recall)

f_score:  0.8030347063952155
precision:  0.8727889346987632
recall:  0.74360499702558


In [44]:
sample_doc_id = list(pred_docs.keys())[0]
test_docs[sample_doc_id]

In [47]:
sample_doc_id

'16362077'

In [85]:
mylist = []
for i in pred_docs[sample_doc_id].ents:
    print(i)
    str_entity = str(i).split(",")
    mylist.append((str_entity[1].strip(),str_entity[-1][:-1].strip()))

(T1, PR_EXT, (16, 21), Sam68)
(T2, CHEBI_SO_EXT, (22, 25), RNA)
(T3, CHEMINF_GO_EXT, (26, 33), Binding)
(T4, CHEBI_PR_EXT, (34, 41), Protein)
(T5, NCBITaxon, (51, 55), Mice)
(T6, PR_EXT, (98, 101), Src)
(T7, GO, (126, 133), mimitos)
(T8, PR_EXT, (145, 150), Sam68)
(T9, PR_EXT, (157, 159), KH)
(T10, SO_EXT, (159, 164), -type)
(T11, CHEBI_SO_EXT, (165, 168), RNA)
(T12, CHEMINF_GO_EXT, (169, 176), binding)
(T13, CHEBI_PR_EXT, (177, 184), protein)
(T14, GO, (208, 216), regulate)
(T15, GO, (236, 250), RNA metabolism)
(T16, GO_PATO_EXT, (265, 276), physiologic)
(T17, PR_EXT, (339, 344), Sam68)
(T18, SO_EXT, (345, 349), null)
(T19, NCBITaxon, (350, 354), mice)
(T20, GO_SO_EXT, (369, 382), recombination)
(T21, PR_EXT, (389, 394), Sam68)
(T22, SO_EXT, (394, 395), −)
(T23, SO_EXT, (396, 397), −)
(T24, NCBITaxon, (398, 402), mice)
(T25, UBERON_EXT, (419, 423), bone)
(T26, SO_EXT, (466, 475), wild-type)
(T27, UBERON_EXT, (497, 501), bone)
(T28, UBERON_EXT, (559, 563), bone)
(T29, PR_EXT, (591, 596

In [84]:
mylist

[('PR_EXT', 'Sam68'),
 ('CHEBI_SO_EXT', 'RNA'),
 ('CHEMINF_GO_EXT', 'Binding'),
 ('CHEBI_PR_EXT', 'Protein'),
 ('NCBITaxon', 'Mice'),
 ('PR_EXT', 'Src'),
 ('GO', 'mimitos'),
 ('PR_EXT', 'Sam68'),
 ('PR_EXT', 'KH'),
 ('SO_EXT', '-type'),
 ('CHEBI_SO_EXT', 'RNA'),
 ('CHEMINF_GO_EXT', 'binding'),
 ('CHEBI_PR_EXT', 'protein'),
 ('GO', 'regulate'),
 ('GO', 'RNA metabolism'),
 ('GO_PATO_EXT', 'physiologic'),
 ('PR_EXT', 'Sam68'),
 ('SO_EXT', 'null'),
 ('NCBITaxon', 'mice'),
 ('GO_SO_EXT', 'recombination'),
 ('PR_EXT', 'Sam68'),
 ('SO_EXT', '−'),
 ('SO_EXT', '−'),
 ('NCBITaxon', 'mice'),
 ('UBERON_EXT', 'bone'),
 ('SO_EXT', 'wild-type'),
 ('UBERON_EXT', 'bone'),
 ('UBERON_EXT', 'bone'),
 ('PR_EXT', 'Sam68'),
 ('SO_EXT', '−'),
 ('SO_EXT', '−'),
 ('NCBITaxon', 'mice'),
 ('SO_EXT', 'wild-type'),
 ('PR_EXT', 'Sam68'),
 ('SO_EXT', '−'),
 ('SO_EXT', '−'),
 ('NCBITaxon', 'mice'),
 ('PR_EXT', 'Sam68'),
 ('SO_EXT', '−'),
 ('SO_EXT', '−'),
 ('UBERON_EXT', 'bone'),
 ('UBERON', 'bone m'),
 ('CL', 'stroma

In [5]:
import os
data_dir = 'brat/'
sec_doc_ids = [data_dir+fname.split('.')[0]+".ann" for fname in os.listdir(data_dir)]
sec_doc_ids= np.unique(sec_doc_ids)

In [6]:
#load file
def txt_strtonum_feed(filename):
    print(filename)
    data = []
    flag=True
    with open(filename, 'r', encoding='UTF-8') as f:#with auto call close()
        line = f.readline()
        
        while line:
            read_data = line.split('\n')
            if flag:
                flag=False
                line = f.readline()
                continue
                
            read_data = read_data[0].split('\t')
            #"brat/12585968.ann"末尾存在不符合规则的内容
            if len(read_data) < 3:
                line = f.readline()
                continue
            
            first = read_data[1].split()[0].split(":")[0]
            second = read_data[1].split()[0]
            third=read_data[2]
            data.append((first,second,third))
            line = f.readline()
        return data

In [7]:
data_ = txt_strtonum_feed("brat/12585968.ann")

brat/12585968.ann


In [9]:
data_ = []
for i in sec_doc_ids:
    data_+=(txt_strtonum_feed(i))

brat/11319941.ann
brat/11532192.ann
brat/11597317.ann
brat/11604102.ann
brat/11897010.ann
brat/12079497.ann
brat/12546709.ann
brat/12585968.ann
brat/12925238.ann
brat/14609438.ann
brat/14611657.ann
brat/14624252.ann
brat/14675480.ann
brat/14691534.ann
brat/14723793.ann
brat/14737183.ann
brat/15005800.ann
brat/15018652.ann
brat/15040800.ann
brat/15061865.ann
brat/15070402.ann
brat/15207008.ann
brat/15238161.ann
brat/15314655.ann
brat/15314659.ann
brat/15320950.ann
brat/15328533.ann
brat/15328538.ann
brat/15345036.ann
brat/15492776.ann
brat/15550985.ann
brat/15560850.ann
brat/15588329.ann
brat/15615595.ann
brat/15619330.ann
brat/15630473.ann
brat/15676071.ann
brat/15760270.ann
brat/15784609.ann
brat/15819996.ann
brat/15836427.ann
brat/15850489.ann
brat/15876356.ann
brat/15882093.ann
brat/15917436.ann
brat/15921521.ann
brat/15938754.ann
brat/16026622.ann
brat/16027110.ann
brat/16098226.ann
brat/16103912.ann
brat/16109169.ann
brat/16110338.ann
brat/16121255.ann
brat/16121256.ann
brat/16216

In [10]:
uni_data_ = list(set(data_))

In [11]:
uni_data_

[('GO', 'GO:0006914', 'autophagy'),
 ('GO_EXT', 'GO_EXT:0015267', 'channel pore'),
 ('SO_EXT', 'SO_EXT:sequence_variant_entity', 'Variant'),
 ('SO_EXT', 'SO_EXT:0000346', 'loxP ... sites'),
 ('CHEBI', 'CHEBI:52071', 'Dextran'),
 ('GO_EXT', 'GO_EXT:oligomeric_macromolecular_complex', 'oligomeric'),
 ('GO', 'GO:0061138', 'branching morphogenesis of epithelium'),
 ('GO_EXT', 'GO_EXT:0022836', 'gated channel'),
 ('CHEBI', 'CHEBI:41218', 'β-mercaptoethanol'),
 ('GO', 'GO:0065007', 'governs'),
 ('GO', 'GO:0006281', 'repairing'),
 ('CL', 'CL:0000084', 'T cell'),
 ('GO', 'GO:0021982', 'development of ... pineal gland'),
 ('UBERON', 'UBERON:0004681', 'vestibular system'),
 ('GO_EXT', 'GO_EXT:0034061', 'DNA polymerase'),
 ('UBERON', 'UBERON:0009201', 'nephric duct'),
 ('UBERON', 'UBERON:0005483', 'thymic lobes'),
 ('GO', 'GO:0001658', 'Branching morphogenesis of ... ureteric bud'),
 ('GO_EXT', 'GO_EXT:biological_mover_or_transporter', 'transporter'),
 ('UBERON', 'UBERON:0000004', 'nasal'),
 ('GO

In [13]:
with open("unidata.txt", 'w', encoding='UTF-8') as f:
    for i in uni_data_:
        data_str = "__label__"+i[1]+" "+i[0]+" "+i[2]+"\n"
        f.write(data_str)
        

In [15]:
len(data_)

159137